# Processing scraped jobs

Once job details and descriptions are scraped a few more things need to happen:

* compare today's vs. yesterday's jobs
* when new job comes in, add day tracker (days = 0)
* if job not new, days += 1

**Import some libraries**

In [2]:
import pandas as pd
import numpy as np

# plotting pacakges
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
%matplotlib inline

# ignore matplotlib/seaborn deprecation warning... it just looks ugly
import warnings
warnings.filterwarnings("ignore", message="The 'normed' kwarg is deprecated, and has been ")

# assortment of other libraries
import re
import time
from nltk.corpus import stopwords
import string
from collections import Counter
import pandas as pd
import numpy as np

In [39]:
df_old = pd.read_csv("./data/glassdoor-df-06-05-2018.csv", encoding="ISO-8859-1")
df_new = pd.read_csv("./data/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")

In [40]:
print(df_old.shape)
df_old.head(n=5)

(901, 4)


,company,location,position,time
0,Agoda,"San Francisco, CA",Data Science Manager - Bangkok,NaN
1,Glassdoor,"Mill Valley, CA","Lead Data Scientist, Decision Science",5.0
2,StubHub,"San Francisco, CA",Revenue Modeling & Data Science Lead,8.0
3,NinthDecimal,"Mountain View, CA",Data Scientist,NaN
4,Zoosk,"San Francisco, CA","Director, Data Science",7.0


In [41]:
print(df_new.shape)
df_new.head(n=5)

(905, 4)


,company,location,position,time
0,Entytle,"Palo Alto, CA",Data Analyst,8.0
1,Castlight Health,"San Francisco, CA",Data Science Intern,NaN
2,Datastax,"Santa Clara, CA",Data Science Intern,NaN
3,Agoda,"San Francisco, CA",Data Science Manager - Bangkok,NaN
4,Palo Alto Networks,"Santa Clara, CA",Machine Learning Software Engineer (Threat Int...,4.0


In [47]:
test1 = ('hi','there'), ('hello', 'friend')
test2 = ('hello', 'again'), ('hi','there')

for i in range(len(test1)):
    if test1[i] in test2:
        print("match found!")
    else:
        print("no match")

match found!
no match


In [52]:
df_old.shape

(901, 4)

In [53]:
df_new.shape

(905, 4)

In [93]:
def compare_days(df_old, df_new):
    
    new = []
    check = 0

    old_jobs = [(x,y) for x,y in zip(df_old['company'],df_old['position'])]
    new_jobs = [(x,y) for x,y in zip(df_new['company'],df_new['position'])]

    for i in range(len(new_jobs)):
        if new_jobs[i] in old_jobs:
            new.append("No")
            result = "No"
        else:
            new.append("Yes")
            result = "Yes"
            
        if check < 20 and result=="Yes":
            print(new_jobs[i])
            check += 1
    
    counts = Counter(new)
    
    df_new["New"] = pd.Series(new).values
    
    print(counts)
    
    return df_new

**Parse the job descriptions for key word counts [COMPLETE!]**

In [ ]:
def parse_top_terms_glassdoor(job_descriptions):
    
    stop_words = set(stopwords.words('english'))

    job_tokens = []
    
    for job in job_descriptions:
        soup = BeautifulSoup(job, 'html.parser')
        words = soup.find('div', id='JobDescriptionContainer')
        words = words.text.strip().replace('\n', ' ').translate(string.punctuation)
        words = re.sub(r'[^\w\s]', '', words)
        filtered_words = list(filter(lambda w: not w in stop_words, words.lower().split()))
        job_tokens = job_tokens + filtered_words
        
    counted_words = Counter(job_tokens)
    
    return(counted_words)

In [94]:
df_test = compare_days(df_old, df_new)

('YapStone', 'Data Scientist')
('eBay Inc.', 'Sr. Data Scientist')
('Reali', 'Product Manager')
('eBay Inc.', 'Data Scientist, Marketing')
('Esri', 'Data Scientist')
('Axia Technologies', 'Senior Product Manager')
('Trianz', 'Python Full Stack Developer')
('realtor.com', 'Sr / Staff Software Engineer')
('Entytle', '\r\n\r\n                        Data Analyst\r\n')
('Match', 'Growth Marketer (Growth Hacker)')
('Ancestry', 'Quality Engineer - Data Science Engineering Team')
('Esri', 'Data Science Product Engineer - ArcGIS API for Python')
('eBay Inc.', 'Staff Data Scientist')
('M Theory Solutions', 'Data Scientist / Predictive Modeler')
('eBay Inc.', 'Data Scientist/Applied Researcher')
('Accenture', 'Data Scientist Associate Principal')
('Metromile', 'Director of Analytics')
('eBay Inc.', 'Senior Data Scientist - GBH')
('Metromile', 'Senior Product Manager, Post-Purchase')
('eBay Inc.', 'Data Scientist 2')
Counter({'No': 519, 'Yes': 386})


In [96]:
df_old[df_old['company'] == 'eBay Inc.']

,company,location,position,time
231,eBay Inc.,"San Francisco, CA",Head of Data Science & Engineering,12.0
247,eBay Inc.,"San Francisco, CA",Revenue Modeling & Data Science Lead,8.0


In [95]:
df_new[df_new['company'] == 'eBay Inc.']

,company,location,position,time,New
10,eBay Inc.,"San Francisco, CA",Sr. Data Scientist,6.0,Yes
15,eBay Inc.,"San Jose, CA","Data Scientist, Marketing",10.0,Yes
40,eBay Inc.,"San Jose, CA",Staff Data Scientist,17.0,Yes
48,eBay Inc.,"San Jose, CA",Data Scientist/Applied Researcher,NaN,Yes
52,eBay Inc.,"San Jose, CA",Senior Data Scientist - GBH,24.0,Yes
56,eBay Inc.,"San Jose, CA",Data Scientist 2,28.0,Yes
376,eBay Inc.,"San Jose, CA",Global Director Display & Social,10.0,Yes
443,eBay Inc.,"San Jose, CA","Manager, Clients Insights & Analytics",23.0,Yes
538,eBay Inc.,"San Francisco, CA",Revenue Modeling & Data Science Lead,10.0,No
787,eBay Inc.,"San Jose, CA",Staff Data Scientist,17.0,Yes


In [74]:
df_test.head(n=20)

,company,location,position,time,New
0,Entytle,"Palo Alto, CA",Data Analyst,8.0,No
1,Castlight Health,"San Francisco, CA",Data Science Intern,NaN,No
2,Datastax,"Santa Clara, CA",Data Science Intern,NaN,No
3,Agoda,"San Francisco, CA",Data Science Manager - Bangkok,NaN,No
4,Palo Alto Networks,"Santa Clara, CA",Machine Learning Software Engineer (Threat Int...,4.0,No
5,Plume,"Palo Alto, CA",Data Science Engineer (Data Scientist),27.0,No
6,YapStone,"Walnut Creek, CA",Data Scientist,NaN,Yes
7,"Quora, Inc.","Mountain View, CA",Data Scientist,4.0,No
8,Castlight Health,"Mountain View, CA",Senior Member of Technical Staff - Data Science,14.0,No
9,Affirm,"San Francisco, CA",Data Scientist,26.0,No


In [86]:
Counter(df_test[(df_test['time'].notnull()) & (df_test['New']=="Yes")]['time'].tolist())

Counter({0.0: 6,
         1.0: 3,
         4.0: 10,
         5.0: 1,
         6.0: 4,
         7.0: 9,
         8.0: 19,
         9.0: 12,
         10.0: 26,
         11.0: 4,
         12.0: 3,
         13.0: 6,
         14.0: 9,
         15.0: 20,
         16.0: 19,
         17.0: 22,
         18.0: 7,
         20.0: 1,
         21.0: 14,
         22.0: 18,
         23.0: 6,
         24.0: 12,
         25.0: 5,
         27.0: 4,
         28.0: 9})